In [2]:
from keras import layers
from keras.models import Sequential
import keras_tuner as kt
from tensorflow.keras import optimizers

In [4]:
def build_model(hp):
    
    model = Sequential()

    model.add(layers.Conv2D(32, (3,3), activation = "relu", input_shape = (150,150,3)))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), activation = "relu"))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), activation = "relu"))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), activation = "relu"))
    model.add(layers.MaxPooling2D((2,2)))


    model.add(layers.Flatten())


    model.add(layers.Dense(units=hp.Choice("layer size", [128, 256, 512]), activation = "relu")) # 64, 128, 256, 512
    model.add(layers.Dense(1, activation = "sigmoid"))

    model.compile(loss = "binary_crossentropy",
              optimizer = optimizers.RMSprop(learning_rate = 1e-4),
              metrics = ["acc"])
    
    return model

In [5]:
keras_tuner = kt.RandomSearch(build_model,
                              objective = "val_acc",
                              overwrite = True,
                              max_trials = 3,
                              directory = "models/parameter_tuning",
                              project_name = "parameter_tuning")

In [6]:
train_directory = "dataset/catdog/train"
validation_directory = "dataset/catdog/validation"

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(train_directory,
                                                    target_size = (150,150),
                                                    batch_size = 20,
                                                    class_mode = "binary")

validation_generator = train_datagen.flow_from_directory(validation_directory,
                                                         target_size = (150,150),
                                                         batch_size = 20,
                                                         class_mode = "binary")

Found 1999 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [9]:
keras_tuner.search_space_summary()
keras_tuner.search(train_generator, validation_data = validation_generator, epochs=25)

Trial 3 Complete [00h 13m 11s]
val_acc: 0.7329999804496765

Best val_acc So Far: 0.753000020980835
Total elapsed time: 00h 41m 18s
INFO:tensorflow:Oracle triggered exit


NameError: name 'tuner' is not defined

In [10]:
best_model = keras_tuner.get_best_models()[0]